## Uploading many lenses

In [ ]:
import os
import sys
import json
import requests
from requests.auth import HTTPBasicAuth
import base64

# Optional (if reading from a .csv file)
import numpy as np

# Optional (if coordinates not in decimal degrees)
from astropy import units as u
from astropy.coordinates import SkyCoord

### 1. Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example, in an open shell terminal type: <code>export SLED_USERNAME=<your_SLED_username></code> and <code>export SLED_PASSWORD=<your_SLED_password></code>. You can also include these two commands in your shell profile file (e.g. ~/.bash_profile in Unix/Linux/Mac) or in an environment file.

In [ ]:
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')
if (not username) or (not password):
    print('You have not set your SLED username and password environment variables.')

### 2. Define a list of lens dictionaries

The minimum information required to upload a lens is the RA, DEC, and an image. Coordinates should be in decimal degrees and in the J2000 system. To convert from ra/dec in hours-minutes/degrees-minutes, you can use the astropy coords functionality. The lens image can be in <code>.png, .jpg, or .jpeg</code> format. It is recommended to restrict access to the uploaded lenses by setting <code>'access_level'='PRI'</code> (or just omitting this field). Additional fields can be added to each lens, as described in the [SLED guide](https://sled.amnh.org/sled_guide/#object-specs-lens).

In [ ]:
# A template lens object. Only ra, dec, and mugshot are required, the rest is optional:
lens_tmp = {
    'ra': 0.0,  # Decimal field, up to 5 decimal places, between 0 and 360
    'dec': 0.0,  # Decimal field, up to 5 decimal places, between -90 and 90
    'name': '',  # String, max_length=100
    'score': 0,  # Decimal number, between 0 and 3
    'mugshot': '',  # The name of the image file, this will be replaced by the actual image data further down
    'flag': 'CANDIDATE', # String, pre-defined choices, see guide
    'lens_type': ['GALAXY'], # List of Strings, pre-defined choices, see guide
    'access_level': 'PRI'
}


# Here we have the RA, DEC, and an image file name as lists:
ras = [0.1,0.2] # dummy values
decs = [-10.1,-10.2] # dummy values
fnames = ['lens01.png','lens02.png'] # dummy values

## If reading from a .csv file use the following command:
#ras,decs,fnames = np.genfromtxt('random_file.csv',dtype=None,skip_header=1,usecols=[0,3,4],delimiter=',',encoding=None,unpack=True)

# And a path where our images are stored:
path_to_img = 'some/dummy/path/'



# We define a list of lens objects:
new_lenses = []
for i in range(0,len(ras)):
    lens = lens_tmp.copy()

    lens['mugshot'] = path_to_img + fnames[i]
    lens['ra'] = round(ras[i],5)
    lens['dec'] = round(decs[i],5)

    ## If the RA and DEC are not in decimal degrees then use the following lines instead:
    #c1 = SkyCoord(ras[i], unit=(u.hourangle, u.deg))
    #c2 = SkyCoord(decs[i], unit=(u.hourangle, u.deg))
    #lens['ra'] = round(c1,5)
    #lens['dec'] = round(c2,5)
    
    if not os.path.isfile(lens['mugshot']):
        print(i,': NO image found with this filename: ',lens['mugshot'])
        sys.exit()
    else:
        new_lenses.append(lens)

        
print('Lenses to be injected: ',len(new_lenses))
if len(new_lenses)>0:
    print('First lens in the list looks like this: ')
    print(new_lenses[0])
else:
    print('No lenses to inject!')
    sys.exit()

### 3. Now send request to the database

Use the requests package to send a POST request to the relevant API endpoint. The *'mugshot'* field for each lens is replaced by the actual image data in base64 encoding. There is no need to change the following lines.


In [ ]:
for lens in new_lenses:
    with open(lens['mugshot'], "rb") as image_file:
        lens['mugshot'] = base64.b64encode(image_file.read())

print("Uploading to the SLED server...")
r = requests.post("https://sled.amnh.org/api/upload-lenses/",
                  data=json.dumps(new_lenses),
                  headers={'Content-Type':'application/json','User-Agent':'SLED-api/1.0'},
                  auth=HTTPBasicAuth(username,password)
                  )
dbquery = json.loads(r.text)

### 4. Understanding the response

There are three possible types of response from the SLED API:
- *errors*: your uploaded sample contains errors, e.g. the RA,DEC are not in the right format, etc.
- *further action required*: if duplicates are found or if public (PUB) lenses are uploaded there will be some further action required.
- *success*: the sample of new lenses was uploaded successfully

In [ ]:
print(dbquery)